In [2]:
import pandas as pd 
import yfinance as yf
import math
import numpy as np 

#List of stocks/etfs to be used 
tickers_list = ['CRM', 'NVDA', 'ADBE', 'GS', 'PG', 'SMCI', 'NEE']
tickers_list.sort()
etf = ['SPY','IWM','DIA']

#Different type of data pulls for Table 1
data = yf.download(tickers_list, period='10y')#General All data
ACdata = yf.download(tickers_list, period = '3mo')['Adj Close'] #Adjusted Close 3 month trail
etf_ticker_data = yf.download(tickers_list + etf, period='1y')['Adj Close'] #Adjusted Close 12 month trail including ETFs
Datahigh = yf.download(tickers_list, period='1y')['High']
Datalow = yf.download(tickers_list, period='1y')['Low']

[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed


In [3]:
##Portfolio Weight
Adj_Closing_Prices= data['Adj Close'].iloc[-1]
total_ACP = Adj_Closing_Prices.sum()

portfolio_weight = []

for Prices in Adj_Closing_Prices:
    portfolio_weight_raw = (Prices/total_ACP)*100
    portfolio_weight.append(round(portfolio_weight_raw,2))

#Portfolio Weight Equally Distributed? 
n = len(tickers_list)
weight = 1/n


In [4]:

#Annualized Volatility 

returns = ACdata.pct_change().dropna() 
volatility_raw = returns.std()*(np.sqrt(252))*100 
annualized_volatility = round(volatility_raw,2)


In [5]:

#Function to use for each ETF to compare to Beta(Tickers/stocks)
def against_etf(etf_symbol):
    beta_values = []

    for ticker in tickers_list:
        # Extract daily returns for the current stock and ETF
        stock_returns = etf_ticker_data[ticker].pct_change().dropna()
        etf_returns = etf_ticker_data[etf_symbol].pct_change().dropna()

        # Calculate covariance and variance
        cov_matrix = np.cov(stock_returns, etf_returns, ddof=0)
        stock_etf_cov = cov_matrix[0, 1]
        etf_variance = np.var(etf_returns, ddof=0)

        # Calculate beta
        beta = round((stock_etf_cov / etf_variance),2)

        beta_values.append(beta)

    return beta_values
    

# Calculate beta for each ETF
Beta_SPY = against_etf('SPY')
Beta_IWM = against_etf('IWM')
Beta_DIA = against_etf('DIA')

In [6]:
#Average Weekly Drop Down 
avg_weekly_dd = []

for ticker in tickers_list:
    highsum = Datahigh[ticker].sum()
    lowsum = Datalow[ticker].sum()
    
    high = highsum/52
    low = lowsum/52

    awdcalc = ((low - high)/high)*100
    avg_weekly_dd.append(round(awdcalc , 2))


In [7]:
#Same code differnt data function (note .max and .min)/ Maximum Weekly Drawdown 
max_weekly_dd = []
for ticker in tickers_list:
    highsummax = Datahigh[ticker].max()
    lowsummin = Datalow[ticker].min()
    
    highmax = highsummax/52
    lowmin = lowsummin/52

    awdcalchm = ((lowmin - highmax)/highmax)*100 
    max_weekly_dd.append(round(awdcalchm, 2))


In [8]:
#Total Return 
Total_Return = []
for ticker in tickers_list:
    datatr = yf.download(ticker, period='10y')  
    beginning_value = datatr['Adj Close'].iloc[0]
    ending_value = datatr['Adj Close'].iloc[-1]

    return_value = ((ending_value - beginning_value)/beginning_value)*100
    Total_Return.append(round(return_value,2))

#Annualized Return dependent on Total Return 
Annualized_Return = []
for i in Total_Return:
   AR = (i/10)*100
   Annualized_Return.append(round(AR,2))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
#Creating a table 
Table1 = {'Ticker' : tickers_list ,
          'Portfolio Weight' : portfolio_weight, 
          'Portfolio Weight Equally Weighted':weight,
          'Annualized Volatility': annualized_volatility,
          'Beta against SPY' : Beta_SPY,
          'Beta against IWM': Beta_IWM ,
          'Beta against DIA': Beta_DIA, 
          'Average Weekly Drawdown':avg_weekly_dd,
          'Maximum Weekly Drawdown': max_weekly_dd,
          'Total Return': Total_Return,
          'Annualized Total Return': Annualized_Return}

df1 = pd.DataFrame(Table1)

df1

,Ticker,Portfolio Weight,Portfolio Weight Equally Weighted,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return
ADBE,ADBE,26.17,0.142857,27.38,1.52,0.94,1.52,-2.59,-51.80,881.71,8817.1
CRM,CRM,10.41,0.142857,22.78,1.33,0.91,1.38,-2.47,-46.96,290.64,2906.4
GS,GS,16.61,0.142857,20.16,0.92,0.78,1.22,-1.99,-26.14,145.28,1452.8
NEE,NEE,2.94,0.142857,25.31,0.79,0.53,0.95,-2.06,-35.70,269.44,2694.4
NVDA,NVDA,22.32,0.142857,38.90,2.13,1.31,1.85,-3.57,-78.49,11815.84,118158.4
PG,PG,7.49,0.142857,12.97,0.49,0.30,0.65,-1.37,-22.86,157.66,1576.6
SMCI,SMCI,14.07,0.142857,73.03,1.37,0.99,1.26,-5.72,-85.92,1925.26,19252.6
